In [6]:
import os
import time
import pyautogui
import random
from dotenv import load_dotenv
import pyperclip
from web3 import Web3
from openai import OpenAI
import requests

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
from src.towns_actions.create_new_town import create_new_town
from src.towns_actions.get_daily_points import get_daily_points
from src.towns_actions.get_info import get_connected_wallet
from src.towns_actions.join_town import join_free_town, join_paid_town
from src.towns_actions.logining import login_twitter, login_google
from src.towns_actions.open_random import open_random_town
from src.towns_actions.open_towns import open_towns
from src.towns_actions.reset_caches import reset_caches
from src.towns_actions.update_towns import update_towns
from src.towns_actions.write_message import write_n_messages

In [10]:
from src.towns_profile_manager import TownsProfileManager

In [12]:
load_dotenv()

DOLPHIN_API = os.getenv("DOLPHIN_API")
TOWNS_LINK = os.getenv("TOWNS_LINK")
OPENAI_API = os.getenv("OPENAI_API")

In [53]:
# towns_profile = TownsProfileManager(profile_id=577170716, anty_type="dolphin", login_with="twitter")
towns_profile = TownsProfileManager(profile_id="kwgrk9t", anty_type="adspower", login_with="twitter")

In [55]:
# open Profile
towns_profile.open_profile()

SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:40497
from session not created: This version of ChromeDriver only supports Chrome version 135
Current browser version is 134.0.6998.36
Stacktrace:
#0 0x58c56454ce9e <unknown>
#1 0x58c563ffcb76 <unknown>
#2 0x58c564039012 <unknown>
#3 0x58c564037f68 <unknown>
#4 0x58c56402e3db <unknown>
#5 0x58c56407d743 <unknown>
#6 0x58c56407cce6 <unknown>
#7 0x58c56406e3e3 <unknown>
#8 0x58c56403bcfd <unknown>
#9 0x58c56403c921 <unknown>
#10 0x58c56451e561 <unknown>
#11 0x58c56452170d <unknown>
#12 0x58c5645211db <unknown>
#13 0x58c564521b75 <unknown>
#14 0x58c56450be3b <unknown>
#15 0x58c564521f3c <unknown>
#16 0x58c5644f4e75 <unknown>
#17 0x58c56453cc78 <unknown>
#18 0x58c56453ce9c <unknown>
#19 0x58c56454ba3c <unknown>
#20 0x70f4a969caa4 <unknown>
#21 0x70f4a9729c3c <unknown>


In [31]:
# open Towns
login_need = open_towns(towns_profile)
login_need

2025-04-13 17:21:47.018 | INFO     | src.towns_actions.open_towns:open_towns:28 - Profile_id: kwgrk9t. Successfully OPENED main page!


False

In [18]:
# twitter login
if login_need:
    login_twitter(towns_profile)

# google login
if login_need:
    login_google(towns_profile)

In [20]:
# # Reset Cache
# reset_caches(driver)

In [22]:
# Get Connected Wallet
get_connected_wallet(towns_profile)

2025-04-02 17:37:17.069 | INFO     | src.towns_actions.get_info:get_connected_wallet:28 - Profile_id: 577164824. Wallet successfully retrieved!


'0xc4093E8F393d10F08D1754d3D5372d327Cb4fbd7'

In [19]:
# open random town
open_random_town(towns_profile)

2025-04-02 21:09:56.423 | INFO     | src.towns_actions.open_random:open_random_town:33 - Profile_id: 577170716. Successfully OPENED random town!


'https://app.towns.com/t/0x9faf3b0c6185bd2ef701a40b5d67062e93a076a5/channels/209faf3b0c6185bd2ef701a40b5d67062e93a076a50000000000000000000000'

In [28]:
# get daily points
get_daily_points(towns_profile)

2025-04-02 17:41:02.208 | INFO     | src.towns_actions.get_daily_points:get_daily_points:45 - Profile_id: 577164824. Daily points are successfully farmed!


In [29]:
# get daily points
get_daily_points(towns_profile)

2025-04-02 17:41:04.496 | INFO     | src.towns_actions.get_daily_points:get_daily_points:48 - Profile_id: 577164824. Cooldown for daily points!


In [27]:
# create new free town
create_new_town(towns_profile, "free", cost=0)

2025-04-02 17:40:51.382 | INFO     | src.towns_actions.create_new_town:create_new_town:117 - Profile_id: 577164824. Town of type free successfully created


'https://app.towns.com/t/0x85f0aa8f0fdfa38f1042bc2807eaa064d4f6503a'

In [23]:
# create new dynamic price town
create_new_town(towns_profile, "dynamic", cost=0)

2025-04-02 17:38:28.686 | INFO     | src.towns_actions.create_new_town:create_new_town:117 - Profile_id: 577164824. Town of type dynamic successfully created


'https://app.towns.com/t/0xf98ca09b34273d17b3d5f5969c4dc59ef8fbaf19'

In [26]:
# create new state price town
create_new_town(towns_profile, "state", cost=0.01)

2025-04-02 17:39:46.053 | INFO     | src.towns_actions.create_new_town:create_new_town:117 - Profile_id: 577164824. Town of type state successfully created


'https://app.towns.com/t/0xba614116009f47d3cb33abeb3d9df26a5bbc482f'

In [36]:
# click on update town button
update_towns(towns_profile)

2025-04-02 17:50:18.489 | INFO     | src.towns_actions.update_towns:update_towns:30 - No element 'Update Towns'


In [38]:
# join free town
town_link = "https://app.towns.com/t/0xdc443716b9c203ff3ee9f3631182bfedf205dac8/"
join_free_town(towns_profile, town_link)

2025-04-02 17:51:25.702 | INFO     | src.towns_actions.join_town:join_free_town:50 - Profile_id: 577164824. Successfully JOINED into free town!


In [40]:
# join paid town
town_link = "https://app.towns.com/t/0x8c5fd1c169c539e244260ba7df63f824d1303ce2"
join_paid_town(towns_profile, town_link, upper_limit=0.99)

Not enough funds


In [44]:
# write a message
town_link = "https://app.towns.com/t/0xdc443716b9c203ff3ee9f3631182bfedf205dac8/" # free_town
write_n_messages(towns_profile, town_link, 4, 10)

2025-04-02 17:54:03.361 | INFO     | src.towns_actions.write_message:write_n_messages:62 - Profile_id: 577164824. Successfully WRITTEN MESSAGE!
2025-04-02 17:54:20.560 | INFO     | src.towns_actions.write_message:write_n_messages:62 - Profile_id: 577164824. Successfully WRITTEN MESSAGE!
2025-04-02 17:54:38.003 | INFO     | src.towns_actions.write_message:write_n_messages:62 - Profile_id: 577164824. Successfully WRITTEN MESSAGE!
2025-04-02 17:54:56.476 | INFO     | src.towns_actions.write_message:write_n_messages:62 - Profile_id: 577164824. Successfully WRITTEN MESSAGE!


In [34]:
towns_profile.close_profile()

In [2]:
a = list()

In [24]:
from src.logic import get_random_town_link
get_random_town_link(towns_profile, "DYNAMIC")

In [88]:
from loguru import logger

In [90]:
def get_connected_wallet(towns_profile: TownsProfileManager):
    # wait till open the page. find user-profile-button
    user_profile_button_element = WebDriverWait(towns_profile.driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//span[@data-testid='user-profile-button']")))

    # check if profile_box is already opened
    check_elements = towns_profile.driver.find_elements(By.XPATH, "//span[@data-testid='upload-image-container']")
    if not check_elements:
        # click user-profile-button if Profile box is not opened
        user_profile_button_element.click()

    # find image_profile_element
    image_profile_element = WebDriverWait(towns_profile.driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//span[@data-testid='upload-image-container']")))

    # find profile_box_element
    profile_box_element = image_profile_element.find_element(By.XPATH, "..").find_element(By.XPATH, "..").find_element(
        By.XPATH, "..")

    # check if there is already wallet available
    wallet_elements = profile_box_element.find_elements(By.XPATH, "//span[contains(text(), '0x')]")
    if len(wallet_elements) == 1:
        return retrieve_wallet_with_validation(towns_profile, copy_connected_wallet_in_profile_box)
    else:
        # find and click Linked Wallets button
        linked_wallets_element = WebDriverWait(towns_profile.driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Linked Wallets')]")))
        linked_wallets_element.click()

        return retrieve_wallet_with_validation(towns_profile, copy_connected_wallet_in_wallets)


def retrieve_wallet_with_validation(towns_profile, copy_wallet_function):
    # retrieve connected wallet until 2 consecutive wallets are identical
    prev_copied_wallet = None
    while True:
        copied_wallet = copy_wallet_function(towns_profile)
        if copied_wallet == prev_copied_wallet:
            towns_profile.wallet = copied_wallet

            logger.info(f"Profile_id: {towns_profile.profile_id}. Wallet successfully retrieved!")
            return copied_wallet
        else:
            prev_copied_wallet = copied_wallet



def copy_connected_wallet_in_wallets(towns_profile: TownsProfileManager):
    # find element with Town Wallet <p>
    towns_wallet_text_element = WebDriverWait(towns_profile.driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//p[contains(text(), 'Towns Wallet')]")))
    # find and click on button that copy text
    copy_wallet_element = towns_wallet_text_element.find_element(By.XPATH, "..").find_element(By.XPATH, "span")
    copy_wallet_element.click()

    return pyperclip.paste()


def copy_connected_wallet_in_profile_box(towns_profile: TownsProfileManager):
    # find image_profile_element
    image_profile_element = WebDriverWait(towns_profile.driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//span[@data-testid='upload-image-container']")))

    # find profile_box_element
    profile_box_element = image_profile_element.find_element(By.XPATH, "..").find_element(By.XPATH, "..").find_element(
        By.XPATH, "..")

    # find and click on button that copy text
    copy_wallet_element = profile_box_element.find_element(By.XPATH, "//span[contains(text(), '0x')]")
    copy_wallet_element.click()

    return pyperclip.paste()

In [92]:
get_connected_wallet(towns_profile)

2025-04-10 09:26:31.349 | INFO     | __main__:retrieve_wallet_with_validation:41 - Profile_id: kwgrk9t. Wallet successfully retrieved!


'0x9a43abBB10ee4d7BE0Ed0Ba14AD05CaB854a2811'

In [ ]:
# open random channel START

In [180]:
def open_random_channel(driver):
    # find all possible channels
    channel_elements = driver.find_elements(By.XPATH, "//span[contains(@data-testid, '-channel')]")
    
    # choose some channel
    channel_to_open = random.randint(0, len(channel_elements)-1)
    
    # open this channel
    channel_elements[channel_to_open].click()
    
    # wait untill it is loaded
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Send a message to ')]")))

open_random_channel(driver)

In [ ]:
# open random channel END

In [ ]:
if action['action'] == 'write_message':
    # Wait at the barrier until all profiles reach their write_message
    print(f"{profile.name} reached write_message, waiting for others...")
    barrier.wait()
    
    # Share params with other profiles
    with lock:
        shared_params[profile.name] = action['params']
    
    # Wait until all params are shared
    barrier.wait()
    
    # Execute write_message with shared params
    all_params = {**action['params'], 'shared': shared_params}
    profile.execute_action({'action': 'write_message', 'params': all_params})

In [ ]:
barrier = threading.Barrier(len(profiles))

In [145]:
def arrange_shuffled_actions(actions):
    write_actions = []
    other_actions = []
    
    for action in actions:
        if action["action"] == "write_message":
            write_actions.append(action)
        else:
            other_actions.append(action)

    shuffled_other_actions = other_actions.copy()
    random.shuffle(shuffled_other_actions)

    return shuffled_other_actions + write_actions

In [ ]:
def run_profile(profile: Profile, actions: List[Dict[str, any]], barrier: threading.Barrier):
    # Shuffle actions for this profile
    shuffled_actions = arrange_shuffled_actions(actions)
    
    for action in shuffled_actions:
        if action['action'] == 'write_message':
            # Wait at the barrier until all profiles reach their write_message
            barrier.wait()
            
            # Execute write_message with shared params
            profile.execute_action({'action': 'write_message', 'params': all_params})
        else:
            # Execute non-write actions immediately
            profile.execute_action(action)

In [ ]:
def run_profile_group(profile_group: List[TownsProfileManager]):
    barrier = threading.Barrier(len(profiles))  # Barrier for write_message synchronization
    
    # Start threads for each profile
    threads = []
    for profile in profiles:
        t = threading.Thread(target=run_profile, args=(profile, actions, barrier, shared_params, lock))
        threads.append(t)
        t.start()
    
    # Wait for all threads to complete
    for t in threads:
        t.join()
    
    print("All profiles completed.")

In [30]:
group_of_n = 3

In [32]:
# LOGIC START

In [34]:
# create group of profiles to run in concurrent way
profile_groups = generate_profile_groups(group_of_n)
# parse actions to perform
parsed_actions = parse_actions('actions.txt')

Group 1 (size 3):
[<src.towns_profile_manager.TownsProfileManager object at 0x75e0eb88dc70>, <src.towns_profile_manager.TownsProfileManager object at 0x75e0eb883470>, <src.towns_profile_manager.TownsProfileManager object at 0x75e0eb882a20>]


Group 2 (size 1):




NameError: name 'parse_actions' is not defined

In [199]:
# create group of profiles to run in concurrent way
profile_groups = generate_profile_groups(group_of_n)
# parse actions to perform
parsed_actions = parse_actions('actions.txt')

for profile_group in profile_groups:
    # Barrier for write_message synchronization
    barrier = threading.Barrier(len(profile_group))
    
    # Start threads for each profile
    threads = []
    for profile in profiles:
        t = threading.Thread(target=run_actions, args=(profile, actions, barrier))
        threads.append(t)
        t.start()
    
    # Wait for all threads to complete
    for t in threads:
        t.join()
    
    print("All profiles completed.")

In [18]:
from numpy import floor, ceil
import pandas as pd

def generate_profile_groups(group_of_n=5):
    # Read the CSV file
    df = pd.read_csv("profiles.csv")
    
    # Shuffle the DataFrame rows
    df = df.sample(frac=1).reset_index(drop=True)
    
    # Generate random group sizes
    group_sizes = []
    remaining = len(df)
    
    while remaining > 0:
        group_size = random.randint(group_of_n - int(floor(group_of_n * 0.25)), group_of_n + int(ceil(group_of_n * 0.25)))
        group_size = min(group_size, remaining)  # Ensure it doesn't exceed available rows
        group_sizes.append(group_size)
        remaining -= group_size
    
    # Split into groups
    groups = []
    start = 0
    for size in group_sizes:
        group = []
        for _, profile_info in df.iloc[start:start + size].iterrows():
            group.append(TownsProfileManager(
                profile_id=profile_info["profile_id"], 
                anty_type=profile_info["anty_type"], 
                login_with=profile_info["login_with"]
            ))
        groups.append(group)
        start += size
    
    # Example: Print group sizes
    for i, group in enumerate(groups):
        print(f"Group {i+1} (size {len(group)}):")
        print(group)
        print("\n")

In [20]:
generate_profile_groups(group_of_n=5)

Group 1 (size 4):
[<src.towns_profile_manager.TownsProfileManager object at 0x75e0eb8830e0>, <src.towns_profile_manager.TownsProfileManager object at 0x75e0ebbd4e90>, <src.towns_profile_manager.TownsProfileManager object at 0x75e0ebaeba10>, <src.towns_profile_manager.TownsProfileManager object at 0x75e0eb883f50>]




In [ ]:
def run_actions(towns_profile: Profile, actions: List[Dict[str, any]], barrier: threading.Barrier):
    # Shuffle actions for this profile
    shuffled_actions = arrange_shuffled_actions(actions)
    
    # 1st action
    towns_profile.open_profile()
    
    # 2nd action
    login_need = open_towns(towns_profile)
    
    # login if needed
    if login_need:
        if towns_profile.login_with == "TWITTER":
            login_twitter(towns_profile)
        elif towns_profile.login_with == "GOOGLE":
            login_google(towns_profile)
        else:
            print("Wrong login type")
            
        get_connected_wallet(towns_profile)
        
    for action in parsed_actions:
        #create action
        if action["action"] == "create_free_channel":
            if random.random() < action["param"]["chance"]:
                create_new_town(towns_profile, "free")
                
        if action["action"] == "create_dynamic_channel":
            if random.random() < action["param"]["chance"]:
                create_new_town(towns_profile, "dynamic")
                
        if action["action"] == "create_paid_channel":
            if random.random() < action["param"]["chance"]:
                create_new_town(towns_profile, "state", action["param"]["cost"])
    
        # join town actions
        if action["action"] == "join_free_channel":
            if random.random() < action["param"]["chance"]:
                town_link = get_random_town_link("free")
                join_free_town(towns_profile, town_link)
                
        if action["action"] == "join_dynamic_channel":
            if random.random() < action["param"]["chance"]:
                town_link = get_random_town_link("dynamic")
                join_paid_town(towns_profile, town_link, upper_limit=action["param"]["cost_limit"])
            
        if action["action"] == "join_paid_channel":
            if random.random() < action["param"]["chance"]:
                town_link = get_random_town_link("state")
                join_paid_town(towns_profile, town_link, upper_limit=action["param"]["cost_limit"])
        
        # write message action
        if action["action"] == "write_message":
            if random.random() < action["param"]["chance"]:                
                # Wait at the barrier until all profiles reach their write_message
                barrier.wait()
                
                if action["param"]["link"]:
                    town_link = action["param"]["link"]
                else:
                    town_link = get_random_town_link(action["param"]["town_type"])
                    
                # Execute write_message with shared params
                town_link = get_random_town_link(action["param"]["link"])
                profile.execute_action({'action': 'write_message', 'params': all_params})
                
                write_message(towns_profile, town_link)
        
        if action["action"] == "get_daily_points":
            if random.random() < action["param"]["chance"]:
                get_daily_points(towns_profile)

In [ ]:
# LOGIC END

In [ ]:
# write review START

In [351]:
def write_review(driver):
    # open review tab
    review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Review This Town')]")))
    review_element.click()

    # click write_review button to open box with writing review
    write_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Write a Review')]")))
    write_review_element.click()

    # find an element in the box
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, ".//*[contains(text(), 'Your Review')]")))
    your_review_element_xpath = get_full_xpath_element(driver, your_review_element)
    
    # find xpath of the review box
    review_box_xpath = your_review_element_xpath[:your_review_element_xpath.rfind("span[") - 1]
    review_box_element = driver.find_element("xpath", review_box_xpath)

    # find text_area
    review_element = review_box_element.find_element("xpath", ".//textarea")
    # generate review
    review_text = generate_review()
    # send text to the textarea
    review_element.send_keys(review_text)

    time.sleep(1)
    
    # find rating element
    select_rating_element = driver.find_element("xpath","//*[contains(text(), 'Select your rating')]")
    rating = random.randint(4, 5)
    rating_element = select_rating_element.find_element("xpath", "..").find_elements("xpath", ".//div")[rating - 1]
    # rate town
    rating_element.click()

    time.sleep(1) # sleep between ac
    
    # click Post_review
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Post Review')]")))
    post_review_element.click()
    
    # click on Pay_with_ETH
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Pay with ETH')]")))
    pay_with_ETH_element.click()
    
    # Wait till review will be submitted
    driver.find_element("xpath", "//*[contains(text(), 'Review posted successfully')]")

    time.sleep(1)
    
def send_keys(element, text):
    for letter in text:
        review_element.send_keys(letter)
        time.sleep(random.randint(1, 7)/100)
    
def generate_review():
    return "great town. I guess it will grow and grow"

In [399]:
# write review END

In [2]:
def trim_stacktrace(log: str) -> str:
    """
    Keeps only the first two stacktrace lines that start with '#'.
    """
    lines = log.strip().splitlines()
    trimmed_lines = []
    count = 0

    for line in lines:
        if line.strip().startswith("#"):
            if count < 2:
                trimmed_lines.append(line)
                count += 1
            else:
                break
        else:
            trimmed_lines.append(line)

    return "\n".join(trimmed_lines)


In [4]:
log = """
Stacktrace:
#0 0x652a66e0a29e <unknown>
#1 0x652a668d9de6 <unknown>
#2 0x652a66925685 <unknown>
#3 0x652a66925a81 <unknown>
#4 0x652a6696d9a4 <unknown>
"""

print(trim_stacktrace(log))


Stacktrace:
#0 0x652a66e0a29e <unknown>
#1 0x652a668d9de6 <unknown>
